In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/u/shuhan/projects/vla')

from src.environments.highway_env.dataset import HighwayDataset, collate_fn

In [2]:
import torch
from torch.utils.data import DataLoader

In [5]:
from torch.utils.data import DataLoader

data_folder = '/storage/Datasets/highway_env/highway_fast_v0_dqn_meta_action_5_lanes/rollouts_train'
dataset = HighwayDataset(data_folder)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

In [6]:
len(dataset)

479134

In [7]:
for batch in dataloader:
  break


In [8]:
obs, act, valid_mask = batch
print(obs.shape, act.shape, valid_mask.shape)

torch.Size([2, 30, 5, 5]) torch.Size([2, 30]) torch.Size([2, 30])


In [81]:
%load_ext autoreload
%autoreload 2

# load GPT-2 
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = 'HuggingFaceTB/SmolLM2-135M-Instruct'
hidden_dim = 576

# checkpoint = 'gpt2'
# hidden_dim = 768

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
%load_ext autoreload
%autoreload 2

from src.auto_labeling.highway_env.lane_change import LaneChangeTaskSpec
from src.models.vlas.cont_obs_token_action_cot_unified_token import ContObsTokenActionCOTVLAUnifiedToken

llm_backbone = model
task_spec_func = LaneChangeTaskSpec

obs_dim = 25
num_actions = 5
mlp_layers = 2

loss_weight = {'action': 1.0, 'reconst': 1.0, 'cot': 1.0, 'separator': 1.0, 'rollout_stop': 1.0}
cot_cfg = {'lanes_count': 5, 'max_hop': 4, 'cot_index_mode': 'both'}

cot_mode = 'all'
# cot_mode = 'start'
# cot_mode = 'none'

vla = ContObsTokenActionCOTVLAUnifiedToken(llm_backbone, tokenizer, task_spec_func, obs_dim, num_actions, hidden_dim, mlp_layers, loss_weight, cot_mode, cot_cfg, max_obs_len=50)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
obs.shape

torch.Size([2, 30, 5, 5])

In [88]:
loss_dict, batch_input_embeds = vla.forward(obs, act, valid_mask)

In [89]:
loss_dict

{'separator': tensor(28.5757, grad_fn=<MeanBackward0>),
 'rollout_stop': tensor(30.0580, grad_fn=<MeanBackward0>),
 'action': tensor(27.6343, grad_fn=<MeanBackward0>),
 'cot': tensor(0.9193, grad_fn=<MeanBackward0>),
 'reconst': tensor(0.2950, grad_fn=<DivBackward0>),
 'total': tensor(87.4823, grad_fn=<AddBackward0>)}

In [232]:
batch_flat_labels['rollout_stop']['label']

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [119]:
# stack the batch seq_emds with padding
# record validity mask for each sequence
batch_seq_emds_flat = torch.nn.utils.rnn.pad_sequence(batch_seq_emds, batch_first=True)
batch_seq_emds_valid = torch.zeros_like(batch_seq_emds_flat[..., 0], dtype=torch.bool)
for bidx in range(len(batch_seq_emds)):
  batch_seq_emds_valid[bidx, :batch_seq_emds[bidx].shape[0]] = 1

In [190]:
batch_flat_labels = {type_name: {} for type_name in batch_index_labels[0].keys()}

# flatten action labels
action_labels = batch_index_labels[0]['act']
action_bidx_list = [torch.tensor(bidx).repeat(len(batch_index_labels[bidx]['act']['index'])) for bidx in range(len(batch_index_labels))]
batch_flat_labels['act']['bidx'] = torch.cat(action_bidx_list, dim=0)
batch_flat_labels['act']['tidx'] = torch.cat([torch.tensor(index_labels['act']['index']) for index_labels in batch_index_labels], dim=0)
batch_flat_labels['act']['label'] = torch.cat([torch.tensor(index_labels['act']['label']) for index_labels in batch_index_labels], dim=0)

In [191]:
# flatten cot labels
cot_index_flat, cot_label_flat = [], []

for bidx in range(len(batch_index_labels)):
  cot_index_flat.append(torch.cat(batch_index_labels[bidx]['cot']['index'], dim=0))
  cot_label_flat.append(torch.cat(batch_index_labels[bidx]['cot']['label'], dim=1)[0])

cot_bidx_list = [torch.tensor(bidx).repeat(len(cot_index_flat[bidx])) for bidx in range(len(batch_index_labels))]
batch_flat_labels['cot']['bidx'] = torch.cat(cot_bidx_list, dim=0)
batch_flat_labels['cot']['tidx'] = torch.cat(cot_index_flat, dim=0)
batch_flat_labels['cot']['label'] = torch.cat(cot_label_flat, dim=0)

In [215]:
# flatten cot_cls and rollout_stop labels
for cls_type in ['cot_cls', 'rollout_stop']:
  cls_index_flat, cls_label_flat = [], []
  for bidx in range(len(batch_index_labels)):
    cls_index_flat.append(torch.tensor(batch_index_labels[bidx][cls_type]['index']))
    cls_label_flat.append(torch.tensor(batch_index_labels[bidx][cls_type]['label'], dtype=torch.long))
  cls_bidx_list = [torch.tensor(bidx).repeat(len(cls_index_flat[bidx])) for bidx in range(len(batch_index_labels))]
  batch_flat_labels[cls_type]['bidx'] = torch.cat(cls_bidx_list, dim=0)
  batch_flat_labels[cls_type]['tidx'] = torch.cat(cls_index_flat, dim=0)
  batch_flat_labels[cls_type]['label'] = torch.cat(cls_label_flat, dim=0)

In [155]:
batch_index_labels[bidx]['cot']['label']

[tensor([[ 3844,   379, 15016,   604,    13,  7281, 15016,   513,  4613, 15016,
            362,  4613, 15016,   352,  4613, 15016,   362,    13,  7406,   318,
          15016,   513,    13,  7561,    25,  1210,  1364,    13, 50257]]),
 tensor([[ 3844,   379, 15016,   604,    13,  7281, 15016,   513,  4613, 15016,
            362,  4613, 15016,   352,  4613, 15016,   362,    13,  7406,   318,
          15016,   513,    13,  7561,    25,  1210,  1364,    13, 50257]]),
 tensor([[ 3844,   379, 15016,   513,    13,  7281, 15016,   362,  4613, 15016,
            352,  4613, 15016,   362,    13,  7406,   318, 15016,   362,    13,
           7561,    25,  1210,  1364,    13, 50257]]),
 tensor([[ 3844,   379, 15016,   362,    13,  7281, 15016,   352,  4613, 15016,
            362,    13,  7406,   318, 15016,   352,    13,  7561,    25,  1210,
           1364,    13, 50257]]),
 tensor([[ 3844,   379, 15016,   362,    13,  7281, 15016,   352,  4613, 15016,
            362,    13,  7406,   318, 1

In [154]:
batch_index_labels[bidx]['cot']['index']

[tensor([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
         48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58]),
 tensor([120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133,
         134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
         148]),
 tensor([156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
         170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181]),
 tensor([189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202,
         203, 204, 205, 206, 207, 208, 209, 210, 211]),
 tensor([219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232,
         233, 234, 235, 236, 237, 238, 239, 240, 241]),
 tensor([249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262,
         263, 264, 265, 266, 267, 268]),
 tensor([282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295,
         296, 297, 298, 299, 300, 301])]

In [125]:
action_bidx_list

[tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1])]

In [123]:
torch.tensor(batch_index_labels[0]['act']['index'])

tensor([ 54,  60,  66,  99, 129, 135, 141, 171, 198, 204, 210, 216, 222, 228,
        234, 240, 246, 252, 258, 264, 270, 276, 282, 288, 294, 300, 306, 333])

In [122]:
torch.tensor(batch_index_labels[0]['act']['label'])

tensor([3, 3, 3, 2, 1, 3, 1, 2, 1, 1, 2, 1, 4, 1, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2,
        2, 2, 2, 0])

In [120]:
batch_index_labels[0]['act']['index']

[54,
 60,
 66,
 99,
 129,
 135,
 141,
 171,
 198,
 204,
 210,
 216,
 222,
 228,
 234,
 240,
 246,
 252,
 258,
 264,
 270,
 276,
 282,
 288,
 294,
 300,
 306,
 333]

In [98]:
batch_index_labels[0]['cot_cls']['index']

[26,
 59,
 65,
 71,
 104,
 134,
 140,
 146,
 176,
 203,
 209,
 215,
 221,
 227,
 233,
 239,
 245,
 251,
 257,
 263,
 269,
 275,
 281,
 287,
 293,
 299,
 305,
 311]

In [95]:
batch_index_labels[0]['cot']['index']

[tensor([27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44,
         45, 46, 47, 48, 49, 50, 51, 52]),
 tensor([72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
         90, 91, 92, 93, 94, 95, 96, 97]),
 tensor([105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
         119, 120, 121, 122, 123, 124, 125, 126, 127]),
 tensor([147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
         161, 162, 163, 164, 165, 166, 167, 168, 169]),
 tensor([177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190,
         191, 192, 193, 194, 195, 196]),
 tensor([312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325,
         326, 327, 328, 329, 330, 331])]

In [81]:
batch_seq_emds[0].shape

torch.Size([339, 768])

In [83]:
batch_seq_emds[1].shape

torch.Size([309, 768])

In [78]:
batch_preview_strs[1]

'Goal is to reach Lane 2. Need to go through path Lane 4 -> Lane 3 -> Lane 2 -> Lane 1 -> Lane 2. [BOO] <Obs_0> [EOO] [BOT] Now at Lane 4. Follow Lane 3 -> Lane 2 -> Lane 1 -> Lane 2. Next is Lane 3. Action: turn left.[EOT] [BOA] <Act_0> [EOA] [BOO] <Obs_1> [EOO] [BOA] <Act_1> [EOA] [BOO] <Obs_2> [EOO] [BOA] <Act_2> [EOA] [BOO] <Obs_3> [EOO] [BOA] <Act_3> [EOA] [BOO] <Obs_4> [EOO] [BOA] <Act_4> [EOA] [BOO] <Obs_5> [EOO] [BOA] <Act_5> [EOA] [BOO] <Obs_6> [EOO] [BOA] <Act_6> [EOA] [BOO] <Obs_7> [EOO] [BOA] <Act_7> [EOA] [BOO] <Obs_8> [EOO] [BOA] <Act_8> [EOA] [BOO] <Obs_9> [EOO] [BOA] <Act_9> [EOA] [BOO] <Obs_10> [EOO] [BOT] Now at Lane 4. Follow Lane 3 -> Lane 2 -> Lane 1 -> Lane 2. Next is Lane 3. Action: turn left.[EOT] [BOA] <Act_10> [EOA] [BOO] <Obs_11> [EOO] [BOT] Now at Lane 3. Follow Lane 2 -> Lane 1 -> Lane 2. Next is Lane 2. Action: turn left.[EOT] [BOA] <Act_11> [EOA] [BOO] <Obs_12> [EOO] [BOT] Now at Lane 2. Follow Lane 1 -> Lane 2. Next is Lane 1. Action: turn left.[EOT] [BO

In [14]:
vla.llm_tokenizer.encode('[EOT]')

[50257]

In [71]:
outputs, loss_dict = vla(obs, act, valid_mask)

In [72]:
loss_dict

{'action': tensor(1.6000, grad_fn=<DivBackward0>),
 'obs': tensor(0.8963, grad_fn=<DivBackward0>),
 'reconst': tensor(0.2941, grad_fn=<DivBackward0>),
 'total': tensor(2.7904, grad_fn=<AddBackward0>)}

In [73]:
loss_dict['total'].backward()